In [1]:
import pandas as pd

# Replace with the correct file paths on your local system
train_data = pd.read_csv(r"C:\Users\HARRY JUAN\Desktop\child-mind-institute-problematic-internet-use\train.csv")
test_data = pd.read_csv(r"C:\Users\HARRY JUAN\Desktop\child-mind-institute-problematic-internet-use\test.csv")
data_dict = pd.read_csv(r"C:\Users\HARRY JUAN\Desktop\child-mind-institute-problematic-internet-use\data_dictionary.csv")

In [2]:
actigraphy_data = pd.read_parquet(r"C:\Users\HARRY JUAN\Desktop\child-mind-institute-problematic-internet-use\series_train.parquet")#會花一點時間

觀察資料狀態

In [4]:

print(train_data.head(20))
print(actigraphy_data.head(10))

          id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0   00008ff9                      Fall                5                0   
1   000fd460                    Summer                9                0   
2   00105258                    Summer               10                1   
3   00115b9f                    Winter                9                0   
4   0016bb22                    Spring               18                1   
5   001f3379                    Spring               13                1   
6   0038ba98                      Fall               10                0   
7   0068a485                      Fall               10                1   
8   0069fbed                    Summer               15                0   
9   0083e397                    Summer               19                1   
10  0087dd65                    Spring               11                1   
11  00abe655                      Fall               11                0   
12  00ae59c9

In [5]:
#計算資料缺失程度
missing_data = train_data.isnull().mean() * 100
print(missing_data[missing_data > 0].sort_values(ascending=False))

PAQ_A-PAQ_A_Total                         88.005051
PAQ_A-Season                              88.005051
Fitness_Endurance-Time_Mins               81.313131
Fitness_Endurance-Time_Sec                81.313131
Fitness_Endurance-Max_Stage               81.237374
                                            ...    
Physical-Weight                           22.323232
PreInt_EduHx-computerinternet_hoursday    16.641414
Physical-Season                           16.414141
FGC-Season                                15.505051
PreInt_EduHx-Season                       10.606061
Length: 78, dtype: float64


In [6]:
# #把缺失太嚴重的欄位(attribute)去掉
columns_to_drop = missing_data[missing_data > 50].index
train_data = train_data.drop(columns=columns_to_drop)

print(train_data.head(20))

          id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0   00008ff9                      Fall                5                0   
1   000fd460                    Summer                9                0   
2   00105258                    Summer               10                1   
3   00115b9f                    Winter                9                0   
4   0016bb22                    Spring               18                1   
5   001f3379                    Spring               13                1   
6   0038ba98                      Fall               10                0   
7   0068a485                      Fall               10                1   
8   0069fbed                    Summer               15                0   
9   0083e397                    Summer               19                1   
10  0087dd65                    Spring               11                1   
11  00abe655                      Fall               11                0   
12  00ae59c9

 Data preprocessing 

In [9]:
#剩下採用插補(以下方法為目前做法，若後續因此預測出的SSI不準確需要調整可以採其他方式
# numerical data 用 中位數插補
numerical_columns = [
    'Physical-BMI', 'Physical-Height', 'Physical-Weight', 'CGAS-CGAS_Score', 'PCIAT-PCIAT_Total', 
    'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP', 
    'FGC-FGC_CU', 'FGC-FGC_PU', 'FGC-FGC_SRL', 'FGC-FGC_SRR', 'FGC-FGC_TL', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 
    'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 
    'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 
    'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 
    'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 
    'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20'
]

for col in numerical_columns:
    train_data[col].fillna(train_data[col].median(), inplace=True)

# 類別變數用眾數插補
categorical_columns = [
    'Basic_Demos-Sex', 'PreInt_EduHx-computerinternet_hoursday', 'Basic_Demos-Enroll_Season', 'CGAS-Season', 
    'Physical-Season', 'SDS-Season', 'PreInt_EduHx-Season', 'FGC-Season', 'FGC-FGC_CU_Zone', 'FGC-FGC_PU_Zone', 
    'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL_Zone', 'BIA-Season', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_Frame_num'
]

for col in categorical_columns:
    train_data[col].fillna(train_data[col].mode()[0], inplace=True)

# 把沒有SII(我們想要預測的目標變數)的資料清掉 
train_data = train_data[train_data['sii'].notna()]


missing_values = train_data.isnull().sum()
print(missing_values[missing_values > 0])  # 檢視是否還有缺失值

train_data


Series([], dtype: int64)


,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,Spring,39.0,55.0,Fall,3.0,2.0
1,000fd460,Summer,9,0,Spring,65.0,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
5,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,...,1.0,2.0,1.0,34.0,Summer,40.0,56.0,Spring,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,ff6c2bb8,Fall,8,0,Spring,65.0,Fall,17.139810,52.5,67.2,...,2.0,2.0,1.0,22.0,Fall,41.0,58.0,Fall,2.0,0.0
3954,ff759544,Summer,7,1,Spring,65.0,Summer,13.927006,48.5,46.6,...,3.0,3.0,0.0,33.0,Summer,48.0,67.0,Summer,0.0,1.0
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
